## Analyze elongation time and collision frequencies

In [20]:
import cPickle as pkl
import pandas as pd

In [17]:
files = [
'/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1137_1000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/1000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1027_1000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/2000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1502_2000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1341_2000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/5000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1847_5000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/5000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1721_5000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/10000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180404_0214_10000_ribosomes_1200s.p',
'/home/martin/git/TRSL/results/10000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_2216_10000_ribosomes_1200s.p',
]

To convert from numpypy to numpy:

In [18]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with open(abs_path,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    close(fh)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

Run only once per file:

In [19]:
for filename in files:
    print filename
    #replace(filename, "_numpypy.multiarray", "numpy.core.multiarray")

/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1137_1000_ribosomes_1200s.p
/home/martin/git/TRSL/results/1000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1027_1000_ribosomes_1200s.p
/home/martin/git/TRSL/results/2000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1502_2000_ribosomes_1200s.p
/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1341_2000_ribosomes_1200s.p
/home/martin/git/TRSL/results/5000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according

Data frame at cell level:

In [50]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

Data frame at transcript level:

In [51]:
dft = pd.DataFrame(columns=['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc'])

Data frame at gene level (synthesis rates):

In [52]:
dfg = pd.DataFrame()

Fill data frames:

In [ ]:
for filename in files[0:1]:
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfg.columns), rowvals))
    print row
    dfc = dfg.append(row, ignore_index=True)
    print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    for transcript in results['transcriptome']:
        rowvals = [results['n_ribosomes'], len(results['transcriptome']), transcript.index, transcript.geneID,
                  transcript.init_rate, transcript.length, transcript.tic_toc]
        row = dict(zip(list(dft.columns), rowvals))
        dft.append(row, ignore_index=True)
    print dft.head()
        
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_dict = {gene: (tc[gene][-1] - tc[gene][-6001]) / 300. for gene in tc}
    dfg[header] = pd.DataFrame(efficiency_dict.items(), columns=['gene', 'synthesis_rate'])

/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1137_1000_ribosomes_1200s.p


In [48]:
files

['/home/martin/git/TRSL/results/1000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1137_1000_ribosomes_1200s.p',
 '/home/martin/git/TRSL/results/1000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1027_1000_ribosomes_1200s.p',
 '/home/martin/git/TRSL/results/2000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1502_2000_ribosomes_1200s.p',
 '/home/martin/git/TRSL/results/2000 ribosomes, 60000 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiation rates according to Shah, deltat=0.05s_20180403_1341_2000_ribosomes_1200s.p',
 '/home/martin/git/TRSL/results/5000 ribosomes, 30061 phase transcriptome, varying tRNAs, full exome, no decay, median-enhanced initiati

In [10]:
results = pkl.load(open(path))

In [11]:
results.keys()

['n_tRNA',
 'description',
 'n_ribosomes',
 'timerange',
 'collisions',
 'proteome',
 'transcriptome',
 'time_stamp',
 'duration',
 'peptide_bonds',
 'timecourses',
 'nocollisions']

In [14]:
results['transcriptome'][0]

<translation.MRNA_specific.mRNA_spec instance at 0x7fc8ccec6200>

In [15]:
dir(_)

['__doc__',
 '__init__',
 '__module__',
 'attach_ribosome',
 'attach_ribosome_at_start',
 'detach_ribosome',
 'find_max_free_range',
 'first_position_occupied',
 'geneID',
 'index',
 'init_rate',
 'length',
 'next_range_free',
 'prev_range_free',
 'ribosomes',
 'sequence',
 'termination_condition',
 'tic',
 'tic_toc',
 'toc',
 'translocate_ribosome']

In [37]:
results['timecourses'].keys()

['YBR177C',
 'YIL140W',
 'YLR268W',
 'YLL062C',
 'YLR197W',
 'YMR111C',
 'YJL122W',
 'YLR244C',
 'YNL003C',
 'YBR135W',
 'YBR160W',
 'YNR007C',
 'YHR007C',
 'YPL191C',
 'YKL190W',
 'YLR395C',
 'YLR348C',
 'YBR248C',
 'YNR067C',
 'YNL152W',
 'YKL063C',
 'YFL045C',
 'YFL010C',
 'YJL131C',
 'YJR044C',
 'YJR011C',
 'YGL141W',
 'YMR009W',
 'YOR123C',
 'YJL179W',
 'YNL072W',
 'YDR363W',
 'YLR362W',
 'YPR118W',
 'YPL024W',
 'YKL134C',
 'YKL029C',
 'YPL166W',
 'YNL163C',
 'YBR273C',
 'YHR025W',
 'YHR009C',
 'YFR050C',
 'YLR447C',
 'YGL237C',
 'YIL062C',
 'YDR540C',
 'YOL129W',
 'YDR299W',
 'ribos._bound',
 'YML129C',
 'YHR101C',
 'YNL239W',
 'YDR410C',
 'YNL255C',
 'YGR020C',
 'YNR045W',
 'YML105C',
 'YLL040C',
 'YDR222W',
 'YOR322C',
 'YBR193C',
 'YNL103W',
 'YAL046C',
 'YAL039C',
 'YNR018W',
 'YPL144W',
 'YDL185W',
 'YFR047C',
 'YBR215W',
 'YIL106W',
 'YBR159W',
 'YML127W',
 'YCL030C',
 'YLR188W',
 'YGR084C',
 'YDR286C',
 'YFR018C',
 'YBR069C',
 'YNR060W',
 'YLR355C',
 'YGL157W',
 'YPL128C',

In [40]:
len(results['timecourses']['YBR177C'])

23999